In [4]:
import os
import numpy as np
import librosa
from hmmlearn import hmm
from sklearn.metrics import classification_report
from glob import glob

# 設定資料路徑
DATASET_PATH = "./drone_audio_dataset_sample"  # 音訊樣本，請跟程式碼放在同個資料夾

def extract_features(file_path, n_mfcc=24, add_deltas=True): # MCFF取24個特徵
    y, sr = librosa.load(file_path, sr=44100) #取樣率44.1kHz
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    if add_deltas:
        delta = librosa.feature.delta(mfcc)
        delta2 = librosa.feature.delta(mfcc, order=2)
        mfcc = np.vstack([mfcc, delta, delta2])
    return mfcc.T  # shape: (time, features)

# 建立每個類別的 HMM 模型
def train_models(data_path):
    models = {}
    for label_dir in os.listdir(data_path):
        label_path = os.path.join(data_path, label_dir)
        if not os.path.isdir(label_path):
            continue
        X_all = []
        for wav_file in glob(os.path.join(label_path, "*.wav")): #.wav檔才可以
            features = extract_features(wav_file, n_mfcc=24, add_deltas=True)
            X_all.append(features)
        X_concat = np.vstack(X_all)
        lengths = [len(x) for x in X_all]
        model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=100)
        model.fit(X_concat, lengths)
        models[label_dir] = model
        print(f"Trained HMM for class: {label_dir}")
    return models

# 測試並分類新樣本
def predict(models, file_path):
    features = extract_features(file_path)
    scores = {}
    for label, model in models.items():
        try:
            score = model.score(features)
            scores[label] = score
        except:
            scores[label] = -np.inf
    return max(scores, key=scores.get)

# 測試整體準確率
def evaluate(models, test_path):
    y_true = []
    y_pred = []
    for label_dir in os.listdir(test_path):
        label_path = os.path.join(test_path, label_dir)
        if not os.path.isdir(label_path):
            continue
        for wav_file in glob(os.path.join(label_path, "*.wav")):
            pred = predict(models, wav_file)
            y_pred.append(pred)
            y_true.append(label_dir)
    print(classification_report(y_true, y_pred))

if __name__ == "__main__":
    print("Training models...")
    models = train_models(DATASET_PATH + "/train")
    print("Evaluating models...")
    evaluate(models, DATASET_PATH + "/test")


Training models...
Trained HMM for class: bird
Trained HMM for class: car
Trained HMM for class: drone
Trained HMM for class: plane
Trained HMM for class: rain
Evaluating models...
              precision    recall  f1-score   support

        bird       1.00      1.00      1.00         1
         car       1.00      1.00      1.00         2
       drone       0.50      1.00      0.67         1
       plane       1.00      1.00      1.00         1
        rain       1.00      0.50      0.67         2

    accuracy                           0.86         7
   macro avg       0.90      0.90      0.87         7
weighted avg       0.93      0.86      0.86         7

